In [109]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import cv2
import numpy as np
import os
import random
import sys
import torchvision.models as models
import torch.optim as optim
import logging
import time
import warnings
from tensorboardX import SummaryWriter
import os
import adv_dataset
from adv_dataset import adv_dataset

import contrastive_loss
from contrastive_loss import contrastive_loss
import network
from network import CNN

In [118]:
torch.cuda.set_device(3)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [119]:
transform1 = transforms.Compose([transforms.ToPILImage(),transforms.ToTensor(),transforms.Normalize(mean=(0.485, 0.456, 0.406),std= (0.229, 0.224, 0.225)) ])


In [120]:
test_set =  adv_dataset('./fgsm_test.txt',False,transform1)
testloader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=0)

In [121]:
cnn = models.resnet152(pretrained=True)
#model = CNN(50)

In [122]:
MODEL_PATH = os.path.join('./model_output_50/', 'resnet_simaese.pth')
state_dict = torch.load(MODEL_PATH, map_location="cpu")
#print(cnn.parameters())
#print(state_dict.keys())
state_dict_v2={}
for key in state_dict:
    #print(key)
    #print(key[10:])
    
    
    state_dict_v2[key[10:]]=state_dict[key]


In [123]:
cnn.load_state_dict(state_dict_v2)   

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [124]:
_=cnn.eval()
iter_test = 0
acc_count_1 = 0
acc_count_5 = 0 

In [125]:
for data in testloader:
    net1 = cnn.to(DEVICE)
    #net2 = net2.to(DEVICE)
    
    img1 = data['img1'].to(DEVICE)
    #img1 = img1.float()
    img1 = img1.view(-1,img1.shape[-3],img1.shape[-2],img1.shape[-1])
    img1 = img1.float()
    #print(img1.shape)
    label = data['label'].to(DEVICE)

    label= label.view(-1)
    label = label-1
    output= net1.forward(img1)
    #output = net2.forward(reps)
    softmax_test = nn.functional.softmax(output, dim=1).data
    #print("check the 1 accuracy")
    batch_size = label.size(0)
    values, index = torch.topk(softmax_test,1)
    index = index.t()
    correct1 = index.eq(label.view(1, -1).expand_as(index))
    #print("correct 1 matrix")
    #print(correct1)
    #count = 0
    tot_correct_1 = 0
    iter_test +=1

    for count in np.arange(1):

        #print("check the count")
        #print(count)
        #print(correct1[count,:].view(-1).float().sum(0, keepdim=True))
        tot_correct_1 += correct1[count,:].view(-1).float().sum(0, keepdim=True)
    acc_count_1 += (tot_correct_1/batch_size)
    print(acc_count_1)
    #print(values)
    #print(index)
    values1, index1 = torch.topk(softmax_test,5)
    #batch_size = label_test.size(0)
    #print("batch size")
    #print(batch_size)
    #print("check the 5 accuracy")
    index1 = index1.t()
    correct = index1.eq(label.view(1, -1).expand_as(index1))
    #print(correct)
    #count = 0
    tot_correct = 0
    for count in np.arange(5):

        #print("check the count")
        #print(count)
        #print(correct[count,:].view(-1).float().sum(0, keepdim=True))
        tot_correct += correct[count,:].view(-1).float().sum(0, keepdim=True)
    #net2.train()
    acc_count_5 += (tot_correct/batch_size)
    #print(acc_count_5)
print("final accuracy 1")
print(acc_count_1/iter_test)
print("final accuracy 5")
print(acc_count_5/iter_test)

tensor([0.], device='cuda:3')
tensor([0.], device='cuda:3')
tensor([0.], device='cuda:3')
tensor([0.], device='cuda:3')
tensor([0.], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
tensor([0.2000], device='cuda:3')
